In [4]:
!wget https://www.openslr.org/resources/68/metadata.tar.gz

--2025-05-09 07:31:37--  https://www.openslr.org/resources/68/metadata.tar.gz
Resolving www.openslr.org (www.openslr.org)... 

46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/68/metadata.tar.gz [following]
--2025-05-09 07:31:37--  https://openslr.elda.org/resources/68/metadata.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3886385 (3.7M) [application/x-gzip]
Saving to: ‘metadata.tar.gz’

metadata.tar.gz     100%[===================>]   3.71M  4.47MB/s    in 0.8s    

2025-05-09 07:31:38 (4.47 MB/s) - ‘metadata.tar.gz’ saved [3886385/3886385]



In [6]:
!tar -xzvf /external4/datasets/Mandarin/MAGICDATA/metadata.tar.gz

Usage_Instruction.txt
MAGICDATA_Mandarin_Chinese_Read_Speech.docx
SPKINFO.txt
dev.scp
test.scp
train.scp


In [ ]:
!tar -xzvf /external4/datasets/Mandarin/MAGICDATA/train_set.tar.gz

test/
test/38_5890/
test/38_5890/38_5890_20170918125135.wav
test/38_5890/38_5890_20170918135804.wav
test/38_5890/38_5890_20170918140400.wav
test/38_5890/38_5890_20170918125920.wav
test/38_5890/38_5890_20170918140609.wav
test/38_5890/38_5890_20170918124902.wav
test/38_5890/38_5890_20170918135824.wav
test/38_5890/38_5890_20170918124754.wav
test/38_5890/38_5890_20170918130832.wav
test/38_5890/38_5890_20170918140500.wav
test/38_5890/38_5890_20170918132830.wav
test/38_5890/38_5890_20170918134123.wav
test/38_5890/38_5890_20170918133211.wav
test/38_5890/38_5890_20170918131704.wav
test/38_5890/38_5890_20170918140536.wav
test/38_5890/38_5890_20170918130028.wav
test/38_5890/38_5890_20170918135234.wav
test/38_5890/38_5890_20170918130354.wav
test/38_5890/38_5890_20170918131531.wav
test/38_5890/38_5890_20170918130731.wav
test/38_5890/38_5890_20170918140755.wav
test/38_5890/38_5890_20170918131100.wav
test/38_5890/38_5890_20170918131752.wav
test/38_5890/38_5890_20170918125350.wav
test/38_5890/38_5890

In [2]:
import os
import shutil
import json
import soundfile as sf

# Hard-coded configuration for each split
SPLITS = {
    "train": {
        "scp": "/external4/datasets/Mandarin/MAGICDATA/train.scp",
        "trans": "/external4/datasets/Mandarin/MAGICDATA/train/TRANS.txt",
        "out_dir": "/external4/datasets/Mandarin/MAGICDATA/wavs_train",
        "manifest": "/external4/datasets/Mandarin/MAGICDATA/manifest_train.jsonl"
    },
    "dev": {
        "scp": "/external4/datasets/Mandarin/MAGICDATA/dev.scp",
        "trans": "/external4/datasets/Mandarin/MAGICDATA/dev/TRANS.txt",
        "out_dir": "/external4/datasets/Mandarin/MAGICDATA/wavs_dev",
        "manifest": "/external4/datasets/Mandarin/MAGICDATA/manifest_dev.jsonl"
    },
    "test": {
        "scp": "/external4/datasets/Mandarin/MAGICDATA/test.scp",
        "trans": "/external4/datasets/Mandarin/MAGICDATA/test/TRANS.txt",
        "out_dir": "/external4/datasets/Mandarin/MAGICDATA/wavs_test",
        "manifest": "/external4/datasets/Mandarin/MAGICDATA/manifest_test.jsonl"
    }
}

# Parse speaker metadata from SPKINFO.txt
speaker_info = {}
spkinfo_path = "/external4/datasets/Mandarin/MAGICDATA/SPKINFO.txt"
with open(spkinfo_path, encoding="utf-8") as spk_f:
    next(spk_f)  # skip header
    for line in spk_f:
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        spkid, age, gender, dialect = parts[0], parts[1], parts[2], parts[3]
        speaker_info[spkid] = {
            "age": age,
            "gender": gender.title(),
            "dialect": dialect
        }

for split, cfg in SPLITS.items():
    base_dir = os.path.dirname(cfg["scp"])

    # Read transcript mapping (UtteranceID, SpeakerID, Transcription)
    trans_map = {}
    with open(cfg["trans"], encoding="utf-8") as tf:
        header = tf.readline()
        for line in tf:
            parts = line.strip().split("\t")
            if len(parts) < 3:
                continue
            utt_id, speaker_id, transcript = parts[0], parts[1], parts[2]
            trans_map[utt_id] = {
                "speaker": speaker_id,
                "text": transcript
            }

    # Ensure output directory exists
    os.makedirs(cfg["out_dir"], exist_ok=True)

    # Write manifest
    with open(cfg["manifest"], "w", encoding="utf-8") as manifest_fh:
        with open(cfg["scp"], encoding="utf-8") as scp_fh:
            for line in scp_fh:
                parts = line.strip().split()
                if len(parts) < 2:
                    continue
                utt_id, raw_path = parts[0], parts[1]

                # Build absolute source path: strip leading 'wav/' then join base_dir
                if raw_path.startswith("wav/"):
                    rel = raw_path[len("wav/"):]
                else:
                    rel = raw_path
                src_path = os.path.join(base_dir, rel)

                # Destination path under split-specific wavs folder
                dest_path = os.path.join(cfg["out_dir"], utt_id)

                # Copy audio file
                try:
                    shutil.copy(src_path, dest_path)
                except FileNotFoundError:
                    print(f"Warning: Source file not found: {src_path}")
                    continue

                # Compute duration
                try:
                    info = sf.info(dest_path)
                    duration = info.duration
                except Exception as e:
                    print(f"Warning: Could not read duration for {dest_path}: {e}")
                    duration = None

                # Lookup transcript and speaker metadata
                meta = trans_map.get(utt_id, {})
                speaker_id = meta.get("speaker")
                spk_meta = speaker_info.get(speaker_id, {})

                entry = {
                    "audio_filepath": dest_path,
                    "duration": duration,
                    "dialect": spk_meta.get("dialect", ""),
                    "gender": spk_meta.get("gender", ""),
                    "age": spk_meta.get("age", ""),
                    "text": meta.get("text", "")
                }
                manifest_fh.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"[{split}] Manifest: {cfg['manifest']} | Audio → {cfg['out_dir']}")


[train] Manifest: /external4/datasets/Mandarin/MAGICDATA/manifest_train.jsonl | Audio → /external4/datasets/Mandarin/MAGICDATA/wavs_train
[dev] Manifest: /external4/datasets/Mandarin/MAGICDATA/manifest_dev.jsonl | Audio → /external4/datasets/Mandarin/MAGICDATA/wavs_dev
[test] Manifest: /external4/datasets/Mandarin/MAGICDATA/manifest_test.jsonl | Audio → /external4/datasets/Mandarin/MAGICDATA/wavs_test
